# Enzyme Stability Prediction

Last updated: 4 Jan 2023

## 0. Import and fix input data

In [1]:
import zipfile

In [2]:
with zipfile.ZipFile("/home/jovyan/Data/novozymes-enzyme-stability-prediction.zip", 'r') as zip_ref:
    zip_ref.extractall("/home/jovyan/Data")

In [3]:
import numpy as np
import pandas as pd

In [4]:
train_1 = pd.read_csv("/home/jovyan/Data/train.csv")
train_2 = pd.read_csv("/home/jovyan/Data/train_updates_20220929.csv")

test_set = pd.read_csv("/home/jovyan/Data/test.csv")

In [5]:
#train_2.loc[pd.isna(train_2["tm"]) == False,:]

In [6]:
train = pd.merge(left = train_1, right = train_2, left_on = "seq_id", right_on = "seq_id", how = "left")
train = train.sort_values(by = "seq_id")

In [7]:
train["protein_sequence"] = np.where(pd.isna(train["protein_sequence_y"]) == False, train["protein_sequence_y"], train["protein_sequence_x"])

train["pH"] = np.where(pd.isna(train["pH_y"]) == False, train["pH_y"], train["pH_x"])

train["tm"] = np.where(pd.isna(train["tm_y"]) == False, train["tm_y"], train["tm_x"])

In [8]:
train = train[["seq_id","protein_sequence","pH","tm"]]

In [9]:
train

,seq_id,protein_sequence,pH,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5
...,...,...,...,...
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7


In [10]:
train.shape

(31390, 4)

In [11]:
test_set.shape

(2413, 4)

In [12]:
pd.concat([train,test_set], axis = "index")

,seq_id,protein_sequence,pH,tm,data_source
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,NaN
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,NaN
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,NaN
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,NaN
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,NaN
...,...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8.0,NaN,Novozymes
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8.0,NaN,Novozymes
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8.0,NaN,Novozymes
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8.0,NaN,Novozymes


In [13]:
protein_length = []

for i in train["protein_sequence"]:
    protein_length.append(len(i))
    
protein_length = pd.DataFrame(protein_length, columns = ["length of protein sequence"])

In [14]:
protein_length.describe()

,length of protein sequence
count,31390.000000
mean,447.669513
std,640.728935
min,5.000000
25%,197.000000
50%,336.000000
75%,523.000000
max,32767.000000


## 1. Feature engineering

In [15]:
from sklearn.preprocessing import OneHotEncoder

In [16]:
import sklearn
sklearn.__version__

'1.2.0'

In [17]:
encoder = OneHotEncoder(sparse_output = True,
                        drop = "first") # to avoid perfect multicollinearity

In [18]:
#X1 = encoder.fit_transform(train[["protein_sequence"]])
X1 = encoder.fit_transform(pd.concat([train,test_set], axis = "index")[["protein_sequence"]])
X1 = X1[:len(train)]
X1

<31390x31393 sparse matrix of type '<class 'numpy.float64'>'
	with 31389 stored elements in Compressed Sparse Row format>

In [19]:
import scipy

In [20]:
X2 = train[["pH"]]
X2

,pH
0,7.0
1,7.0
2,7.0
3,7.0
4,7.0
...,...
31385,7.0
31386,7.0
31387,7.0
31388,7.0


In [21]:
X = scipy.sparse.hstack([X1,X2])
X

<31390x31394 sparse matrix of type '<class 'numpy.float64'>'
	with 62779 stored elements in COOrdinate format>

## 2. Model training - regressino approach

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, train["tm"], test_size = 0.33, random_state = 20221126)

In [24]:
import xgboost

#### a. Apply gridsearch to optimise hyperparameters

In [25]:
from sklearn.model_selection import GridSearchCV, KFold

In [26]:
kf = KFold(n_splits = 10, shuffle = True, random_state = 20221126)

paras = {
    #'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5, 2, 5],
    #'subsample': [0.6, 0.8, 1.0],
    #'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [3, 4, 5]
}

In [27]:
model_1 = \
GridSearchCV(
    estimator = xgboost.XGBRegressor(),
    param_grid = paras,
    scoring = "neg_mean_squared_error",
    n_jobs = -1,
    refit = True,
    cv = kf,
    verbose = 1
)

In [28]:
model_1.fit(X_train_valid, y_train_valid)

Fitting 10 folds for each of 15 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=10, random_state=20221126, shuffle=True),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=No...
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             n_jobs=-1,
             param_grid={'gamma': [0.5, 1, 1.5, 2, 5], 'max_depth': [3, 4, 5]},
             scoring='neg_mean_squared_error', verbose=1)

In [29]:
model_2 = model_1.best_estimator_
model_2

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.5, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=5, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

#### b. Apply cross-validation to optimise parameters

In [30]:
from sklearn.model_selection import cross_validate

In [31]:
cv_scores = cross_validate(
    estimator = model_2,
    X = X_train_valid, y = y_train_valid,
    scoring = "neg_mean_squared_error",
    n_jobs = -1,
    cv = kf,
    return_estimator = True)

In [32]:
cv_scores["test_score"]

array([-194.48861832, -179.63478749, -181.91744084, -186.67399674,
       -182.73688367, -191.60272355, -171.17056048, -178.28157655,
       -195.26704553, -172.24072938])

In [33]:
print("On validation sets, the best negative MSE is {} at estimator number {}".format(cv_scores["test_score"].max(),cv_scores["test_score"].argmax()))

On validation sets, the best negative MSE is -171.17056048082267 at estimator number 6


In [34]:
model_3 = cv_scores["estimator"][cv_scores["test_score"].argmax()]
model_3

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.5, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=5, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

#### c. Used tuned model to predict

In [35]:
from sklearn.metrics import mean_squared_error

In [36]:
y_pred = model_3.predict(X_test)

print("On the test set, the negative MSE is {}".format(-1 * mean_squared_error(y_pred, y_test)))

On the test set, the negative MSE is -180.54462516096038


#### d. Make predictions on actual test set

In [37]:
train.shape

(31390, 4)

In [38]:
test_set.shape

(2413, 4)

In [39]:
testX1 = encoder.fit_transform(pd.concat([train,test_set], axis = "index")[["protein_sequence"]])
testX1 = testX1[len(train):]
testX1

<2413x31393 sparse matrix of type '<class 'numpy.float64'>'
	with 2413 stored elements in Compressed Sparse Row format>

In [40]:
test_df = scipy.sparse.hstack([testX1, test_set[["pH"]]])

In [41]:
predictions = model_3.predict(test_df)

In [42]:
predictions

array([49.031826, 49.031826, 49.031826, ..., 49.031826, 49.031826,
       49.031826], dtype=float32)

In [43]:
toSubmit = pd.concat([pd.DataFrame(test_set["seq_id"]), pd.DataFrame(predictions)], axis = "columns")
toSubmit = toSubmit.rename(columns = {0:"tm"})

In [44]:
toSubmit.to_csv("to_submit.csv", index = False)